# Prediction of leaderboard score (R)

In [5]:
score_data = read.csv('../input/scores.csv')
score_data[with(score_data, order(score)), ]

,model,score,accuracy,logloss,AUC,f1
3,GradientBoostingClassifier,0.4452,0.8177,6.2962,0.6717,0.507
1,nolearn,0.4566,0.8056,6.7159,0.6711,0.5044
4,ExtraTreesClassifier,0.4729,0.8402,5.5467,0.71,0.5806
5,XGBClassifier,0.4851,0.842,5.4567,0.71,0.5806
13,BaggingClassifier,0.4885,0.8507,5.1568,0.7107,0.5865
2,ensemble of averages,0.4896,NA,NA,NA,NA
11,SVC,0.5336,0.9375,2.1587,0.8745,0.8525
6,SGDClassifier,0.567,0.75,8.6348,0.6545,0.4745
7,cosine_similarity,0.5732,0.7906,7.2333,0.6452,0.4595
8,AdaBoostClassifier,0.6642,0.809,6.596,0.6635,0.4907


In [2]:
lm.fit = lm(score ~ .-model, 
            data      = score_data, 
            na.action = na.omit)
summary(lm.fit)


Call:
lm(formula = score ~ . - model, data = score_data, na.action = na.omit)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.56419 -0.18279  0.03077  0.12517  0.58682 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)
(Intercept) -28.260023 599.415508  -0.047    0.964
accuracy      5.013873 592.710068   0.008    0.994
logloss      -0.007427  17.131772   0.000    1.000
AUC          67.531792  44.685374   1.511    0.181
f1          -40.332209  27.682242  -1.457    0.195

Residual standard error: 0.4096 on 6 degrees of freedom
  (4 observations deleted due to missingness)
Multiple R-squared:  0.4267,	Adjusted R-squared:  0.04443 
F-statistic: 1.116 on 4 and 6 DF,  p-value: 0.4297


In [3]:
slm.fit = step(lm.fit, direction = "both")
summary(slm.fit)

Start:  AIC=-16.3
score ~ (model + accuracy + logloss + AUC + f1) - model

           Df Sum of Sq    RSS     AIC
- logloss   1   0.00000 1.0068 -18.302
- accuracy  1   0.00001 1.0068 -18.302
<none>                  1.0068 -16.302
- f1        1   0.35621 1.3630 -14.970
- AUC       1   0.38326 1.3901 -14.754

Step:  AIC=-18.3
score ~ accuracy + AUC + f1

           Df Sum of Sq    RSS     AIC
- accuracy  1   0.06268 1.0695 -19.638
<none>                  1.0068 -18.302
- f1        1   0.42402 1.4308 -16.436
+ logloss   1   0.00000 1.0068 -16.302
- AUC       1   0.46008 1.4669 -16.162

Step:  AIC=-19.64
score ~ AUC + f1

           Df Sum of Sq    RSS     AIC
<none>                  1.0695 -19.638
- f1        1   0.36771 1.4372 -18.387
+ accuracy  1   0.06268 1.0068 -18.302
+ logloss   1   0.06267 1.0068 -18.302
- AUC       1   0.39750 1.4670 -18.161



Call:
lm(formula = score ~ AUC + f1, data = score_data, na.action = na.omit)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.57080 -0.17375 -0.04049  0.10289  0.59087 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)
(Intercept)   -21.72      12.59  -1.726    0.123
AUC            58.00      33.63   1.724    0.123
f1            -32.72      19.73  -1.658    0.136

Residual standard error: 0.3656 on 8 degrees of freedom
  (4 observations deleted due to missingness)
Multiple R-squared:  0.391,	Adjusted R-squared:  0.2387 
F-statistic: 2.568 on 2 and 8 DF,  p-value: 0.1376


In [4]:
score_data = score_data[with(score_data, order(score)), ]

library(knitr)
kable(score_data, format = "markdown", digits = 4)



|   |model                      |  score| accuracy| logloss|    AUC|     f1|
|:--|:--------------------------|------:|--------:|-------:|------:|------:|
|3  |GradientBoostingClassifier | 0.4452|   0.8177|  6.2962| 0.6717| 0.5070|
|1  |nolearn                    | 0.4566|   0.8056|  6.7159| 0.6711| 0.5044|
|4  |ExtraTreesClassifier       | 0.4729|   0.8402|  5.5467| 0.7100| 0.5806|
|5  |XGBClassifier              | 0.4851|   0.8420|  5.4567| 0.7100| 0.5806|
|13 |BaggingClassifier          | 0.4885|   0.8507|  5.1568| 0.7107| 0.5865|
|2  |ensemble of averages       | 0.4896|       NA|      NA|     NA|     NA|
|11 |SVC                        | 0.5336|   0.9375|  2.1587| 0.8745| 0.8525|
|6  |SGDClassifier              | 0.5670|   0.7500|  8.6348| 0.6545| 0.4745|
|7  |cosine_similarity          | 0.5732|   0.7906|  7.2333| 0.6452| 0.4595|
|8  |AdaBoostClassifier         | 0.6642|   0.8090|  6.5960| 0.6635| 0.4907|
|9  |KNeighborsClassifier       | 1.1870|   0.8142|  6.4161| 0.6620| 0.488